In [2]:
import os

In [ ]:
print(os.getenv("AZURE_OPENAI_API_KEY"))
print(os.getenv("AZURE_OPENAI_ENDPOINT"))
print(os.getenv("OPENAI_API_VERSION"))
print(os.getenv("OPENAI_API_KEY"))

In [1]:
from openai import AzureOpenAI
from openai import OpenAI

In [2]:
# api_version = "2024-07-18"
api_version = "2024-02-15-preview"
azure_endpoint="https://nshan-m3oer8jc-eastus.openai.azure.com/"
azureClient = AzureOpenAI(api_version=api_version, azure_endpoint=azure_endpoint)
# azureClient = AzureOpenAI(azure_endpoint=azure_endpoint)

In [3]:
import minsearch

In [4]:
import json


In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nUsing the FAQ:\nDon’t change the style and page setup (make a copy first if you need to do it for whatever reason)\nUse Search to find what you need\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoint the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [12]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [13]:
response = azureClient.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)
response.choices[0].message.content

"Whether you can enroll in a course after it has started depends on the institution or program's policies. Many institutions allow late enrollments, while others may have strict deadlines. It's best to contact the course administration or registrar's office directly to inquire about your options and any associated fees or requirements."

In [14]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database . Use only the facts from the CONTEXT when answering the QUESTION
If the CONTEXT doesn't contain the answer then output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


In [16]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - What can I do before the course starts?
answer: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terrafo

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
print(len(prompt)/4)
print(prompt)

608.75
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database . Use only the facts from the CONTEXT when answering the QUESTION
If the CONTEXT doesn't contain the answer then output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start workin

In [19]:
response = azureClient.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [5]:
client = OpenAI()

In [37]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [20]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
def llm(prompt):
    response = azureClient.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
print(rag(query))

To run Kafka, you need to ensure your Kafka broker Docker container is working properly. You can check this by using the command `docker ps` to see if it is running. If it is not running, navigate to the folder containing your `docker-compose.yml` file and execute `docker-compose up -d` to start all instances. Additionally, make sure to manage your Kafka environment effectively by using a virtual environment for Python dependencies if applicable.


In [28]:
print(rag('the course has already started, can I still enroll?'))

Yes, you can still enroll in the course after it has started. You are eligible to submit the homework. However, keep in mind that there will be deadlines for turning in the final projects, so it's important not to leave everything until the last minute.


In [31]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200')

In [35]:
es_client.info()

ObjectApiResponse({'name': 'e6f096b7607c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'NQj5UUavQ7yTOZ71cemKCA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [37]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [38]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nUsing the FAQ:\nDon’t change the style and page setup (make a copy first if you need to do it for whatever reason)\nUse Search to find what you need\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoint the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

In [41]:
from tqdm.auto import tqdm

In [42]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)
    

  0%|          | 0/1075 [00:00<?, ?it/s]

In [60]:
def elastic_search(query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [61]:
query='I just discovered the course. Can I still join the course'

In [62]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [63]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [64]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homeworks without registering, but make sure to adhere to the deadlines for the final projects to avoid leaving everything for the last minute.'

In [65]:
rag('What is the use of this course?')

'The context does not provide specific information about the use or objective of the course.'

In [66]:
rag('What are the requirements for this course?')

'The requirements for this course include setting up dependencies and requirements such as a Google Cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git. Additionally, reviewing the prerequisites and syllabus to ensure comfort with these subjects is recommended.'